In [1]:
import data_ingestion.github_data_reader as reader
import data_ingestion.chordpro_parser as parser

In [25]:
songs = reader.read_github_data('Asacri', 'asacriband-chords')
chunks = parser.parse_and_chunk(songs)

In [26]:
from qdrant_client import QdrantClient, models

In [27]:
qd_client = QdrantClient('http://localhost:6333')

In [28]:
dimensionality = 384
model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
collection_name = "asacriband-chords"

In [29]:
if qd_client.collection_exists(collection_name):
    qd_client.delete_collection(collection_name)

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=dimensionality,
        distance=models.Distance.COSINE
    )
)

True

In [30]:
points = []

for i, doc in enumerate(chunks):
    text = doc.content
    text = text.strip()
    vector = models.Document(text=text, model=model_name)


    if (len(text) > 0):
        point = models.PointStruct(id=i, vector=vector, payload={ 'title': doc.title, 'content': doc.content, 'key': doc.key })
        points.append(point)

qd_client.upsert(
    collection_name=collection_name,
    points=points
)

/Users/kasteion/repos/kasteion/ai-bootcamp-project/.venv/lib/python3.13/site-packages/qdrant_client/embed/embedder.py:75: UserWarning: The model sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 now uses mean pooling instead of CLS embedding. In order to preserve the previous behaviour, consider either pinning fastembed version to 0.5.1 or using `add_custom_model` functionality.
  model = TextEmbedding(model_name=model_name, **options)
Exception ignored in: <function tqdm.__del__ at 0x146814040>
Traceback (most recent call last):
  File "/Users/kasteion/repos/kasteion/ai-bootcamp-project/.venv/lib/python3.13/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/Users/kasteion/repos/kasteion/ai-bootcamp-project/.venv/lib/python3.13/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm' object has no attribute 'disp'
Exception ignored in: <function tqdm.__del__ at 0x146814040>
Traceba

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [31]:
from pydantic_ai.messages import FunctionToolCallEvent

class NamedCallback:

    def __init__(self, agent):
        self.agent_name = agent.name

    async def print_function_calls(self, ctx, event):
        # Detect nested streams
        if hasattr(event, "__aiter__"):
            async for sub in event:
                await self.print_function_calls(ctx, sub)
            return

        if isinstance(event, FunctionToolCallEvent):
            tool_name = event.part.tool_name
            args = event.part.args
            print(f"TOOL CALL ({self.agent_name}): {tool_name}({args})")

    async def __call__(self, ctx, event):
        return await self.print_function_calls(ctx, event)


In [32]:
from typing import TypedDict, List

class SearchResult(TypedDict):
    """Represents a single search result entry."""
    title: str
    content: str
    key: str

class SearchTools:
    def vector_search(self, query: str, num_results=15) -> List[SearchResult]:
        """
        Performs a vector search looking for lyrics related with the query.

        Args:
            query (str): The search query string.

        Returns:
            List[SearchResult]: A list of search results. Each result dictionary contains:
                - title (str): The title of the song.
                - content (str): The content of the song that matched.
                - key (str): The key the of the song.
        """
        vector = models.Document(text=query, model=model_name)

        query_points = qd_client.query_points(
            collection_name=collection_name,
            query=vector,
            limit=num_results,
            with_payload=True
        )

        results = []

        for point in query_points.points:
            results.append(point.payload)

        return results

    def get_full_song_by_title(self, title: str) -> SearchResult:
        """
        Retrieve full song by title

        Args:
            title (str): The title of the song
        
        Returns:
            SearchResult: The full song
                - title (str): The title of the song.
                - content (str): The content of the song.
                - key (str): The key the of the song.
        """
        sections = [chunk for chunk in chunks if chunk.title == title]
        if len(sections) == 0:
            return ""
        
        title = sections[0].title
        key = sections[0].key
        content = ' '.join([section.content for section in sections])
        return SearchResult(title = title, content=content, key=key)


In [33]:
tools = SearchTools()

In [34]:
# import re
# section_content = re.sub(r"\[[^\]]+\]", "", songs[1].content)
# print(section_content)
print(tools.get_full_song_by_title("2da de Corintios 5:17"))


{'title': '2da de Corintios 5:17', 'content': 'El que está en Cristo es nueva criatura. Las cosas viejas pasaron ya. He aquí todas son hechas nuevas, Así se inicia el primer amor. Por Cristo vivimos, en Cristo morimos, En Él redimimos el mal que hicimos. Su sangre nos limpia de todo pecado Y así el pasado queda sepultado.', 'key': 'G'}


In [35]:
' '.join([chunk.content for chunk in chunks if chunk.title == 'En Totalidad a Ti'])

'En totalidad a Ti, Rindo alma y corazón, No tengo nada más que dar Tómalo mi buen Señor Tómame en tus brazos Cristo, Toma mi vida en tus manos Y haz de mi lo que Tú quieras, Tómame aquí estoy Tómame aquí estoy'

In [56]:
from pydantic import BaseModel, Field
from typing import List

class Search(BaseModel):
    """keyword used for search and the song titles found"""
    keyword: str = Field(..., description="The keyword searched")
    titles: List[str] = Field(..., description = "A list of song titles from the search results")

class SearchPhase(BaseModel):
    """A list of the searches done by keyword and the relevant songs found"""
    searches: List[Search] = Field(..., description="A list of at 5 searches done to find relevant songs")

class SongList(BaseModel):
    "List of recommended songs with a score"
    title: str = Field(..., description="The title of the recommended song")
    key: str = Field(..., description="Key of the recommended song")

class SongScore(BaseModel):
    """"""
    title: str = Field(..., description="The title of the recommended song")
    key: str = Field(..., description="The key of the recommended song")
    score: int = Field(..., description="The score of the recommended song based on it's relevance")

class RepertoireEvaluation(BaseModel):
    "Final repertoire evaluation"
    songScores: List[SongScore] = Field(..., description="A list of 5 song titles with key and score")
    score: int = Field(..., description="Score for the complete repertoire")
    justification: str = Field(..., description="Justification of the score given to the repertoire")

class Repertoire(BaseModel):
    """The complete repertoir across all phases"""
    searchPhase: List[SearchPhase] = Field(..., description="Search stage report")
    songList: List[SongList] = Field(..., description = "A list of 4-5 recommended songs")
    repertoireEvaluation: RepertoireEvaluation = Field(..., description="Final repertoire evaluation")

In [93]:
from pydantic_ai import Agent

instructions = """
Eres un agente cuyo objetivo es recomendar un listado de canciones 
basadas en un tema en el que el usuario se quiera enfocar.

OBJETIVO

Dado el tema en que el usuario quiere enfocarse, ejecuta búsquedas 
en la base de datos con la letra de las canciones para encontra canciones
relacionadas con el tema. Encuentra 4 o 5 canciones diferentes para usar 
como repertorio de canciones basadas en ese tema.

PROCESO

Fase 1 - Búsqueda inicial

- Tomando el input del usuario crea 5 palabras claves diferentes relacionadas con con ese tema.
- Utiliza `vector_search()` para buscar en la base de datos canciones relacionadas con las palabras clave.
- No te detengas hasta hacer 5 búsquedas.
- NO HAGAS MAS DE 7 BUSQUEDAS.

Fase 2 - Generar lista

Con el output de la Fase 1:

- Genera una lista de 4 a 5 títulos de las canciones y su key

Fase 3 - Evaluar lista

Con el output de la Fase 2:

- Utiliza solo las canciones de la fase 2 y obten la canción completa con `get_full_song_by_title()`
- Evalúa cada canción y asigna un score de 0 a 100 deacuerdo a que tan acertada es con respecto al tema que el usuario quiere
- Evalúa todo el repertorio y asigna un score de 0 a 100 deacuerdo a su relevancia y al score de las canciones.
- Escribe una justificación de score del repertorio
- EN ESTA FASE NO TIENES PERMITIDO volver a ejecutar `vector_search()`
"""

agent_tools = [tools.vector_search, tools.get_full_song_by_title]

agent = Agent(
    name="recomendations",
    instructions=instructions,
    tools=agent_tools,
    model='gpt-4o-mini',
    output_type=Repertoire
)

In [95]:
results = await agent.run(
    user_prompt='Quiero un set de canciones enfocadas en la fe en medio de las pruebas',
    event_stream_handler=NamedCallback(agent)
)

TOOL CALL (recomendations): vector_search({"query": "fe en medio de las pruebas"})
TOOL CALL (recomendations): vector_search({"query": "esperanza en tiempos difíciles"})
TOOL CALL (recomendations): vector_search({"query": "fortaleza en momentos difíciles"})
TOOL CALL (recomendations): vector_search({"query": "superar adversidades"})
TOOL CALL (recomendations): vector_search({"query": "confianza en Dios ante las dificultades"})
TOOL CALL (recomendations): get_full_song_by_title({"title": "Creo en Ti"})
TOOL CALL (recomendations): get_full_song_by_title({"title": "Al Rey de reyes - A Ti sea la gloria"})
TOOL CALL (recomendations): get_full_song_by_title({"title": "Santo"})
TOOL CALL (recomendations): get_full_song_by_title({"title": "Dios de lo Imposible"})
TOOL CALL (recomendations): get_full_song_by_title({"title": "Gloria a ti, Jes\u0015"})
TOOL CALL (recomendations): get_full_song_by_title({"title": "10000 Razones"})
TOOL CALL (recomendations): get_full_song_by_title({"title": "En Je

In [96]:
output = results.output

In [89]:

print("Search Phase: ")
for phase in output.searchPhase:
    for search in phase.searches:
        print(" keyword:", search.keyword)
        for title in search.titles:
            print("    -", title)

Search Phase: 
 keyword: perdón
    - Cristo yo te amo
    - Reina Dios
    - Gracias Cristo
    - Santo
    - El Amor de Dios es maravilloso
 keyword: restauración espiritual
    - Reina Dios
    - Cristo yo te amo
    - Gracias Cristo
    - Santo
    - El Amor de Dios es maravilloso
 keyword: redención
    - Cristo yo te amo
    - Aquel que nos amó
    - Santo, santo, grande eterno Dios
    - En Ti
    - Gracias Cristo
 keyword: sanación
    - Cristo yo te amo
    - El Amor de Dios es maravilloso
    - Dios Poderoso
    - Cuán Grande es Dios
    - Gracias Cristo
 keyword: reconciliación
    - Santo
    - Cordero
    - Cantad alegres al Señor
    - 10000 Razones
    - En Jesucristo fuente de paz


In [90]:
print("Listado:")
for song in output.songList:
    print("  -", song.title, song.key)

Listado:
  - Cristo yo te amo G
  - Reina Dios A
  - Gracias Cristo A
  - Santo D
  - Aquel que nos amó G


In [97]:
print("Te recomiendo tocar las siguientes canciones:")
evaluation = output.repertoireEvaluation
for song in evaluation.songScores:
    print(f"  - '{song.title}' en clave de {song.key} con una relevancia del: {song.score}%")

print()
print(f"Luego de la evaluación este repertorio tiene una relvancia del {evaluation.score}%")

print()
print("Justificación:")
print(evaluation.justification)


Te recomiendo tocar las siguientes canciones:
  - 'Creo en Ti' en clave de A con una relevancia del: 90%
  - 'Al Rey de reyes - A Ti sea la gloria' en clave de A con una relevancia del: 85%
  - 'Santo' en clave de D con una relevancia del: 80%
  - 'Dios de lo Imposible' en clave de C con una relevancia del: 90%
  - '10000 Razones' en clave de C con una relevancia del: 95%

Luego de la evaluación este repertorio tiene una relvancia del 88%

Justificación:
Este repertorio refleja muy bien el tema de la fe en medio de las pruebas. Las canciones seleccionadas hablan de confianza, adoración y esperanza divina, elementos esenciales para enfrentar tiempos difíciles. Las canciones puntuadas más alto abordan directamente el fortalecimiento de la fe y la alabanza a Dios en momentos de dificultad.
